## Overview
This quickstart runs a small QV experiment using the baseline config and prints a concise aggregated table. It also inspects the saved HDF5 and displays a few generated plots.

# Quickstart: Run a Quantum Volume (QV) Experiment End-to-End
This notebook runs a minimal QV experiment using the project's pipeline and shows how to load the configuration, execute, and inspect results and plots.

In [ ]:
# 1) Setup: ensure src is on sys.path and imports work
import sys, os
from pathlib import Path
root = Path.cwd()
src_path = root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
print("Using src path:", src_path)
from spinq_qv.config.schemas import Config
from spinq_qv.experiments.run_qv import run_experiment
from datetime import datetime

In [ ]:
# 2) Load baseline configuration YAML
baseline_yaml = root / "examples" / "configs" / "baseline.yaml"
assert baseline_yaml.exists(), f"Missing baseline config: {baseline_yaml}"
config = Config.from_yaml(baseline_yaml)
# For a quick run, narrow widths and circuits
config.simulation.widths = [2, 3]
config.simulation.n_circuits = 5
config.simulation.n_shots = 200
config.simulation.random_seed = 12345
config.metadata.update({"notebook": "01_quickstart_qv_experiment", "timestamp": datetime.now().isoformat()})
config

In [ ]:
# 3) Run experiment and return aggregated results
output_dir = root / "results" / "quickstart"
output_dir.mkdir(parents=True, exist_ok=True)
aggregated = run_experiment(
    config,
    output_dir,
    seed=config.simulation.random_seed,
    parallel=False,
    n_workers=1,
    enable_profiling=False,
    return_aggregated=True,
 )
aggregated

In [ ]:
# 4) Pretty-print a concise summary table
import pandas as pd
rows = []
for w, data in aggregated.items():
    rows.append({
        "width": int(w),
        "mean_hop": data.get("mean_hop"),
        "ci_lower": data.get("ci_lower"),
        "ci_upper": data.get("ci_upper"),
        "std_hop": data.get("std_hop"),
        "pass_qv": data.get("pass_qv"),
    })
pd.DataFrame(rows).sort_values("width").reset_index(drop=True)

In [ ]:
# 5) Inspect the saved HDF5 (aggregated group)
import h5py
from glob import glob
# Find the run file from the output_dir
h5_files = sorted(glob(str(output_dir / "**/*.h5"), recursive=True))
assert h5_files, f"No HDF5 files found under {output_dir}"
h5_path = h5_files[-1]
print("Using HDF5:", h5_path)
with h5py.File(h5_path, 'r') as f:
    print("Top-level groups:", list(f.keys()))
    assert 'aggregated' in f, "Missing aggregated group"
    widths = list(f['aggregated'].keys())
    print("Widths in aggregated:", widths)
    w0 = widths[0]
    attrs = dict(f['aggregated'][w0].attrs)
    attrs

In [ ]:
# 6) Display generated plots from the run directory
from IPython.display import Image, display
plots = sorted(glob(str(output_dir / "**/plots_*/**/*.png"), recursive=True))
print(f"Found {len(plots)} plot images")
for p in plots[:6]:  # show a few
    display(Image(filename=p))